In [1]:
import GridMLM_tokenizers
from GridMLM_tokenizers import CSGridMLMTokenizer
from models_graph import make_graph_from_input_ids, remove_consecutive_duplicates, remove_out_of_dict_ids, compute_edge_features
from data_utils import CSGridMLMDataset
import torch
from tqdm import tqdm

/home/maximos/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = CSGridMLMTokenizer(fixed_length=256)
chord_features = GridMLM_tokenizers.CHORD_FEATURES
chord_id_features = {tokenizer.vocab[k]: v for k, v in chord_features.items()}
print(chord_features)

{'C:maj': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 'C#:maj': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 'D:maj': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], 'D#:maj': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], 'E:maj': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], 'F:maj': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 'F#:maj': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 'G:maj': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], 'G#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 'A:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 'A#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 'B:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [3]:
print(chord_features['C:maj'])
print(chord_features['D:maj'])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]


In [4]:
print(chord_id_features[7])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]


In [5]:
train_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_train'
val_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_test'
# train_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_train'
# val_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_test'

In [6]:
# train_dataset = GuidedGridMLMDataset(train_dir, tokenizer, 512, frontloading=True)
val_dataset = CSGridMLMDataset(val_dir, tokenizer, frontloading=True, name_suffix='Q4_L80_bar_PC')

Loading data file.


In [7]:
print(val_dataset[0])

{'harmony_ids': [6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'pianoroll': array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'time_signature': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'h_density_complexity': [0, 1, 0, 0, 1, 0, 0, 0]}


In [8]:
chord_id_duplicates_sequence = val_dataset[6]['harmony_ids']

In [9]:
print(chord_id_duplicates_sequence)

[6, 7, 7, 7, 210, 6, 210, 210, 210, 210, 6, 269, 269, 269, 13, 6, 13, 13, 13, 13, 6, 152, 152, 152, 152, 6, 152, 152, 152, 152, 6, 7, 7, 7, 7, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
chord_ids_sequence = remove_consecutive_duplicates( chord_id_duplicates_sequence )
print(chord_ids_sequence)
chord_ids_sequence = remove_out_of_dict_ids( chord_ids_sequence, chord_id_features )
print(chord_ids_sequence)
unique_chord_ids_sequence = list(set(chord_ids_sequence))
print(unique_chord_ids_sequence)
for c in unique_chord_ids_sequence:
    print(c, tokenizer.ids_to_tokens[c], chord_id_features[c])

[6, 7, 210, 6, 210, 6, 269, 13, 6, 13, 6, 152, 6, 152, 6, 7, 6, 7, 1]
[7, 210, 210, 269, 13, 13, 152, 152, 7, 7]
[7, 13, 269, 210, 152]
7 C:maj [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
13 C:7 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
269 A:min [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
210 G:maj [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
152 F:maj [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]


In [11]:
src_id = unique_chord_ids_sequence[0]
dst_id = unique_chord_ids_sequence[1]

src_feat = torch.LongTensor(chord_id_features[src_id])
dst_feat = torch.LongTensor(chord_id_features[dst_id])

feats = compute_edge_features(src_feat, dst_feat)
print(src_id, tokenizer.ids_to_tokens[src_id], src_feat)
print(dst_id, tokenizer.ids_to_tokens[dst_id], dst_feat)
print('Features: ', feats)

7 C:maj tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
13 C:7 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
Features:  tensor([0.0000, 0.7500])


In [12]:
# chord_id_duplicates_sequence = [10,11,12,10]
g = make_graph_from_input_ids(chord_id_duplicates_sequence, chord_id_features, use_probabilities=True)

In [13]:
print(g)

Data(x=[5, 24], edge_index=[2, 9], edge_attr=[9, 2], node_ids=[5], edge_weight=[9])


In [14]:
# make graph dataset

graph_val_dataset = []

for d in tqdm(val_dataset):
    chord_id_duplicates_sequence = d['harmony_ids']
    g = make_graph_from_input_ids(
            chord_id_duplicates_sequence,
            chord_id_features,
            use_probabilities=True
        )
    if g is not None:
        graph_val_dataset.append( g )
    else:
        print('Short sequence: ', chord_id_duplicates_sequence)

100%|██████████| 758/758 [00:00<00:00, 6661.57it/s]

Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
print(chord_id_duplicates_sequence)
# print(g)
print(g.x)
print(g.node_ids)
print(g.edge_index)
print(g.edge_attr)
print(g.edge_weight)

[6, 269, 269, 269, 269, 6, 123, 123, 123, 123, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 6, 269, 7, 7, 7, 6, 66, 66, 66, 66, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0.]])
tensor([ 66, 123, 269,   7])
tensor([[2, 1, 1, 2, 2, 3, 0],
        [1, 1, 2, 2, 3, 0, 1]])
tensor([[0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.5000, 0.5000],
        [0.3333, 0.000

In [16]:
from models_graph import HarmonicGraphEncoder, BilinearDecoder, HarmonicGAE, edge_index_to_dense

In [17]:
hidden_dim = 64

encoder = HarmonicGraphEncoder(hidden_dim=hidden_dim)
decoder = BilinearDecoder(hidden_dim=hidden_dim)
gae = HarmonicGAE(hidden_dim=hidden_dim)


In [18]:
z, z_pooled = encoder(g)

In [19]:
print(z.shape)
print(z_pooled.shape)

torch.Size([4, 64])
torch.Size([1, 64])


In [20]:
x = decoder(z)

In [21]:
print(x)

tensor([[0.3701, 0.0652, 0.1736, 0.3911],
        [0.3361, 0.0342, 0.1494, 0.4802],
        [0.3659, 0.0428, 0.1527, 0.4386],
        [0.3568, 0.0760, 0.1911, 0.3762]], grad_fn=<SoftmaxBackward0>)


In [22]:
y, enc_pooled = gae( g )

In [23]:
print(y)
print(enc_pooled)

tensor([[0.2759, 0.2143, 0.2085, 0.3013],
        [0.3498, 0.0862, 0.1477, 0.4164],
        [0.2343, 0.2683, 0.2220, 0.2754],
        [0.2158, 0.3173, 0.2244, 0.2426]], grad_fn=<SoftmaxBackward0>)
tensor([[-0.0908,  0.2246, -0.8174, -0.7155, -0.0869, -0.3509, -0.6944,  0.3510,
         -0.0363,  1.0447,  0.0377, -0.6090,  0.7410, -0.8304, -0.2873, -1.2179,
          0.1370,  0.2924,  0.3571, -0.6967,  0.8022, -0.3695, -0.0292, -0.3699,
          0.5353,  0.2541,  0.4923, -0.4485, -1.0050, -0.5301, -0.6676,  0.9092,
          0.6453,  1.1616,  1.0562, -0.8148, -0.3798,  0.9913,  0.5769,  0.8617,
          0.8832, -0.1222, -0.5245,  0.8792,  0.0325,  0.4491,  0.0895, -0.1483,
          0.0772,  0.6913, -0.9564, -0.3713,  1.0647,  0.7876, -0.8571, -0.2763,
         -0.7797,  0.1079, -0.5188, -0.9762,  0.8209, -0.7591, -0.5131, -0.7855]],
       grad_fn=<UnsqueezeBackward0>)


In [24]:
target = edge_index_to_dense(g)

loss_fn = torch.nn.KLDivLoss(reduction="batchmean")
loss = loss_fn(torch.log(y + 1e-9), target)

In [25]:
print(target)

tensor([[0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.6667, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333],
        [1.0000, 0.0000, 0.0000, 0.0000]])


In [26]:
print(loss.item())

1.200228214263916


In [27]:
print(len(graph_val_dataset))
g0 = graph_val_dataset[3]
print(g0)

756
Data(x=[4, 24], edge_index=[2, 3], edge_attr=[3, 2], node_ids=[4], edge_weight=[3])


In [28]:
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool

loader = DataLoader(graph_val_dataset, batch_size=32, shuffle=False)

In [29]:
print(len(loader))

24


In [30]:
b = next(iter(loader))

In [31]:
print(b)
print(b.batch)

DataBatch(x=[151, 24], edge_index=[2, 197], edge_attr=[197, 2], node_ids=[151], edge_weight=[197], batch=[151], ptr=[33])
tensor([ 0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  5,
         5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11,
        11, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15,
        15, 16, 16, 16, 16, 16, 17, 17, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23,
        23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27,
        27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30,
        30, 31, 31, 31, 31, 31, 31])


In [35]:
# for training
loss_fn = torch.nn.KLDivLoss(reduction="batchmean")

for batch in loader:
    node_emb, _ = gae.encoder(batch)  # y = [N_total, N_total]
    batch_vec = batch.batch  # [N_total]
    
    batch_loss = 0.0
    
    for g_id in range(batch.num_graphs):
        node_mask = (batch_vec == g_id)
        z = node_emb[node_mask]
        
        probs = gae.decoder(z)
        
        # Get target
        g = batch.get_example(g_id)  # or batch.to_data_list()[g_id]
        target = edge_index_to_dense(g)
        
        batch_loss += loss_fn(torch.log(probs + 1e-9), target)


In [37]:
batch_loss.item()

26.482975006103516

In [ ]:
# for collecting embeddings
embeddings = []

for batch in loader:
    z, _ = encoder(batch)
    graph_embeddings = global_mean_pool(z, batch.batch)
    embeddings.append(graph_embeddings.detach())

embeddings = torch.cat(embeddings, dim=0)

In [ ]:
print(embeddings.shape)

torch.Size([756, 64])
